In [13]:
from snac import SNAC
import torch
import os
import json
from datasets import load_dataset

def new_path(f):
    f = f.replace('.mp3', '.snac')
    splitted = f.split('/')
    base_folder = splitted[0] + '_snac'
    splitted = '/'.join([base_folder] + splitted[1:])
    return splitted

def redistribute_codes(code_list):
    layer_1 = []
    layer_2 = []
    layer_3 = []
    for i in range((len(code_list)+1)//7):
        layer_1.append(code_list[7*i][None])
        layer_2.append(code_list[7*i+1][None]-4096)
        layer_3.append(code_list[7*i+2][None]-(2*4096))
        layer_3.append(code_list[7*i+3][None]-(3*4096))
        layer_2.append(code_list[7*i+4][None]-(4*4096))
        layer_3.append(code_list[7*i+5][None]-(5*4096))
        layer_3.append(code_list[7*i+6][None]-(6*4096))
    
    with torch.no_grad():

        codes = [torch.concat(layer_1)[None], 
            torch.concat(layer_2)[None], 
            torch.concat(layer_3)[None]]
        audio_hat = snac_model.decode(codes)
        return audio_hat.cpu()[0, 0]

In [ ]:
snac_model = SNAC.from_pretrained("hubertsiuzdak/snac_24khz")
snac_model = snac_model.eval().cuda()

In [4]:
dataset = load_dataset('mesolitica/TTS-Combined')['train']
rows = dataset.to_list()

In [7]:
with open(new_path(rows[0]['audio_filename'])) as fopen:
    d = json.load(fopen)

In [15]:
row = torch.tensor(d, device = 'cuda')
row_length = row.size(0)
new_length = (row_length // 7) * 7
trimmed_row = row[:new_length]
trimmed_row = [t - 128266 for t in trimmed_row]
y_ = redistribute_codes(trimmed_row)

In [16]:
import IPython.display as ipd
ipd.Audio(y_, rate = 24000)

In [17]:
ipd.Audio(rows[0]['audio_filename'])